# Exporting BioCantor data models

BioCantor data models can be exported to any of:

1. GenBank
2. GFF3
3. JSON
4. BED (TranscriptInterval and FeatureInterval only).

The JSON representation can be read directly by the `marshmallow` data structures that build the data model.

In [ ]:
from inscripta.biocantor.io.gff3.parser import parse_gff3

gff3 = "tests/data/INSC1006_chrI.gff3"

model = list(parse_gff3(gff3))[0]
parsed = model.to_annotation_collection()

## GFF3

Each of the five interval objects in BioCantor is capable of directly printing to GFF3. If this function is called on a parent object, each child is also recursively called.

In [ ]:
for gff_row in parsed.to_gff():
    print(gff_row)

CM021111.1	BioCantor	gene	16175	18079	.	+	.	ID=9a8b23f6-06e5-72f5-6c06-56b48ce47626;gene_biotype=ncRNA
CM021111.1	BioCantor	transcript	16175	18079	.	-	.	ID=d534e7a9-ba9f-2d18-8365-5e3a319b21bf;Parent=9a8b23f6-06e5-72f5-6c06-56b48ce47626;gene_biotype=ncRNA;transcript_biotype=ncRNA
CM021111.1	BioCantor	exon	16175	18079	.	-	.	ID=exon-d534e7a9-ba9f-2d18-8365-5e3a319b21bf-1;Parent=d534e7a9-ba9f-2d18-8365-5e3a319b21bf;gene_biotype=ncRNA;transcript_biotype=ncRNA
CM021111.1	BioCantor	gene	37462	39103	.	+	.	ID=dfd34bd6-a1df-8bf3-c173-15d5cabd043f;Name=GDH3;gene_name=GDH3;gene_biotype=protein_coding
CM021111.1	BioCantor	transcript	37462	39103	.	+	.	ID=4651a388-3090-2037-e344-ea4a3151eb3a;Parent=dfd34bd6-a1df-8bf3-c173-15d5cabd043f;Name=GDH3;gene_name=GDH3;gene_biotype=protein_coding;transcript_name=GDH3;transcript_biotype=protein_coding
CM021111.1	BioCantor	exon	37462	39103	.	+	.	ID=exon-4651a388-3090-2037-e344-ea4a3151eb3a-1;Parent=4651a388-3090-2037-e344-ea4a3151eb3a;Name=GDH3;gene_name=GDH3;g

### GFF3 with FASTA

In addition to being able to print GFF directly, convenience functions exist to export GFF3 in one go, and optionally include sequence info.

In [ ]:
# this does not work because it was parsed without sequence information
from inscripta.biocantor.io.gff3.writer import collection_to_gff3

with open("/dev/null", "w") as fh:
    collection_to_gff3([parsed], fh, add_sequences=True)

GFF3ExportException: Cannot export FASTA in GFF3 if collection has no associated sequence

In [ ]:
# parse the GFF3 with sequence instead this time and write to disk
from inscripta.biocantor.io.gff3.parser import parse_gff3_embedded_fasta

with open("/dev/null", "w") as fh:
    parsed_with_sequence = [x.to_annotation_collection() for x in parse_gff3_embedded_fasta(gff3)]
    collection_to_gff3(parsed_with_sequence, fh, add_sequences=True)

## JSON

Each object also has a `to_dict()` function, which produces a dict that the `marshmallow` library understands. As a result, the below two operations are identical.

In [ ]:
model.Schema().load(parsed.to_dict()).to_annotation_collection().to_dict() == parsed.to_dict()

True

However, the below is not true, only because the marshmallow schemas are `Ordered`, and so produced `OrderedDict`:

In [ ]:
parsed.to_dict() == model.Schema().dump(model)

False

## BED

BED export is only valid on `TranscriptInterval` and `FeatureInterval` objects, because BED format does not model relationships between rows. All models are exported in `BED12` format.

In [ ]:
for gene_or_feature_collection in parsed:
    for transcript_or_feature in gene_or_feature_collection:
        print(transcript_or_feature.to_bed12())


CM021111.1	16174	18079	None	0	-	0	0	0,0,0	1	1905	0
CM021111.1	37461	39103	GDH3	0	+	37637	39011	0,0,0	1	1642	0
CM021111.1	39518	40772	BDH2	0	+	39518	40772	0,0,0	1	1254	0
CM021111.1	41085	42503	BDH1	0	+	0	0	0,0,0	1	1418	0


## GenBank

All models can be exported to GenBank. GenBank export must be specified to be in either *prokaryotic* or *eukaryotic* flavors. See the document on parsing GenBank files for an explanation of the difference.

GenBank export is problematic for genomes that have multiple isoforms per gene due to the lack of the ability to explicitly define the hierarchical relationship. BioCantor GenBank is always locus sorted, which helps resolve this ambiguity.

GenBank export also the ability to export GenBank files compatible with Inscripta Engineering Portal. This mode of export ensures that there is always a unique `/gene` tag on every feature, and that `CDS` features have a `/translation` tag.

The `organism` and `source` fields can be set by keyword arguments.

In [ ]:
from inscripta.biocantor.io.genbank.writer import collection_to_genbank

with open("/dev/null", "w") as fh:
    collection_to_genbank([parsed], fh)

GenBankExportError: Cannot export GenBank if collections do not have sequence information

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as tmp_dir:
    tmp_file = Path(tmp_dir) / "test.gbk"
    with open(tmp_file, "w") as fh:
        collection_to_genbank(parsed_with_sequence, tmp_file)
    with open(tmp_file, "r") as fh:
        print(fh.read()[:2000])

LOCUS       CM021111.1             50040 bp    DNA              UNK 01-JAN-1980
DEFINITION  GenBank produced by BioCantor.
ACCESSION   CM021111
VERSION     CM021111.1
KEYWORDS    .
SOURCE      None
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     gene            complement(16175..18079)
                     /gene_biotype="ncRNA"
                     /gene
     ncRNA           complement(16175..18079)
                     /gene_biotype="ncRNA"
                     /transcript_biotype="ncRNA"
     gene            37462..39103
                     /gene_name="GDH3"
                     /gene_biotype="protein_coding"
                     /gene_symbol="GDH3"
                     /gene="GDH3"
     CDS             37638..39011
                     /gene_name="GDH3"
                     /gene_biotype="protein_coding"
                     /transcript_name="GDH3"
                     /transcript_biotype="protein_coding"
     gene            39519..40772
                 